In [8]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# Import CSV for TTVFast simulations
df = pd.read_csv(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME\validation\1st_order_TTVFast\lithwick_TTVFast_params.csv")
df.head()

,e1,w1,e2,w2,Delta,j,P1,P2,m1,m2,mu1,mu2,validity1,validity2,valid
0,0.002794,303.782212,0.005116,111.941832,0.013081,4,7,9.455426,613.683372,0.516593,0.001843,0.000002,21557.700803,18.147057,False
1,0.003392,131.310902,0.026566,116.472256,-0.008979,3,7,10.405725,91.639422,375.153540,0.000275,0.001127,2.693084,11.024948,False
2,0.003647,278.084232,0.005244,249.655123,0.007424,5,7,8.814959,169.936265,112.427523,0.000510,0.000338,31.126447,20.592835,False
3,0.068471,240.613521,0.067907,260.675715,-0.005708,3,7,10.440071,24.560454,2.188322,0.000074,0.000007,72.384784,6.449441,False
4,0.026676,308.906024,0.004105,240.631741,0.007924,5,7,8.819331,2394.321680,1.887488,0.007191,0.000006,415.152982,0.327273,False


In [9]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "A2", "B2", "V2", "Pttv2", "phase2", "std2", "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,j,P1,P2,m1,m2,...,phase1,std1,A2,B2,V2,Pttv2,phase2,std2,R2_1,R2_2
0,0.002794,303.782212,0.005116,111.941832,0.013081,4,7,9.455426,613.683372,0.516593,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.003392,131.310902,0.026566,116.472256,-0.008979,3,7,10.405725,91.639422,375.153540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.003647,278.084232,0.005244,249.655123,0.007424,5,7,8.814959,169.936265,112.427523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.068471,240.613521,0.067907,260.675715,-0.005708,3,7,10.440071,24.560454,2.188322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.026676,308.906024,0.004105,240.631741,0.007924,5,7,8.819331,2394.321680,1.887488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# For storing TTVs
df_ttv_cols = ["t0", "ttv0", "t1", "ttv1"]
df_ttv = pd.DataFrame(index=range(len(df)), columns=df_ttv_cols, dtype=object)
df_ttv.head()

,t0,ttv0,t1,ttv1
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [11]:
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):
    # If invalid scenario, skip
    if row['valid'] == False:
        continue
    
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']
    PTTV = 1/np.abs(j/P2 - (j-1)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P2 * 200
    Time = begin_time   # days
    dt = P1 / 100       # days
    Total = max(250, PTTV * 2.5)  # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)

    # Write results to df_fit and df_ttv
    df_ttv.loc[i] = {
        "t0": time0,
        "ttv0": ttv0,
        "t1": time1,
        "ttv1": ttv1
    }
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, *popt_ttv1, std1, r2_0, r2_1]

100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [09:35<00:00, 86.85it/s]


In [12]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.05
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

In [13]:
# Export dataframes
df_ttv.to_csv("lithwick_TTVFast_outputs.csv", index=False)
df_fit.to_csv("lithwick_TTVFast_fits.csv", index=False)